In [1]:
import torch
from PIL import Image
from transformers import AutoModel, CLIPImageProcessor

model = AutoModel.from_pretrained(
    '/cpfs01/shared/llmeval/fangxinyu/hub/models--OpenGVLab--InternViT-6B-448px-V1-5',
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True).cuda().eval()

image = Image.open('/cpfs01/user/fangxinyu/VLMEvalKit/assets/apple.jpg').convert('RGB')

image_processor = CLIPImageProcessor.from_pretrained('/cpfs01/shared/llmeval/fangxinyu/hub/models--OpenGVLab--InternViT-6B-448px-V1-5')

pixel_values = image_processor(images=image, return_tensors='pt').pixel_values
pixel_values = pixel_values.to(torch.bfloat16).cuda()

outputs = model(pixel_values)
print(outputs.last_hidden_state.shape)

FlashAttention is not installed.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([1, 1025, 3200])


In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch

model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384")
processor = AutoProcessor.from_pretrained("google/siglip-so400m-patch14-384")

url = "/cpfs01/user/fangxinyu/VLMEvalKit/assets/apple.jpg"
image = Image.open(url).convert('RGB')

texts = ["a photo of 2 cats"]
inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")
print(inputs.keys())
with torch.no_grad():
    outputs = model(inputs['pixel_values'])
print(outputs)
logits_per_image = outputs.logits_per_image
probs = torch.sigmoid(logits_per_image) # these are the probabilities
print(f"{probs[0][0]:.1%} that image 0 is '{texts[0]}'")


dict_keys(['input_ids', 'pixel_values'])


TypeError: conv2d() received an invalid combination of arguments - got (NoneType, Parameter, Parameter, tuple, str, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!NoneType!, !Parameter!, !Parameter!, !tuple of (int, int)!, !str!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!NoneType!, !Parameter!, !Parameter!, !tuple of (int, int)!, str, !tuple of (int, int)!, int)


: 

In [1]:
import pandas as pd
f = '/cpfs01/user/fangxinyu/LMUData/MMBench_VIDEO_0423_T1998.tsv'
gtdata = pd.read_csv(f, sep='\t')
gt_contents = gtdata.to_dict('records')
gt_contents[0]

{'index': 0,
 'video': 'wZxzBvAgqxc',
 'video_type': 'Sports',
 'question': 'What is the name of the player who scored the first goal in the video?',
 'answer': 'Palmer.',
 'dimensions': "['OCR', 'Attribute Recognition']",
 'video_path': '/cpfs01/shared/llmeval/dhd/MMBench-Video/video/wZxzBvAgqxc.mp4'}

In [8]:
outputs.vision_model_output.last_hidden_state[:, 1:].shape

torch.Size([1, 728, 1152])